# 📊 Evaluación del Modelo Base - DistilBERT

---

## Objetivos del Notebook

1. ✅ **Setup completo**: Cargar configuración, modelo y datos
2. ✅ **Evaluación cuantitativa**: Métricas de clasificación (Accuracy, F1, etc.)
3. ✅ **Análisis cualitativo**: Ejemplos de predicciones correctas e incorrectas
4. ✅ **Distribución de confianza**: Analizar probabilidades del modelo
5. ✅ **Selección de casos**: Identificar ejemplos interesantes para SHAP/LIME

---

**Proyecto:** Interpretabilidad en NLP - Módulo II 

---

In [ ]:
import sys
print("Python ejecutable:", sys.executable)
print("Ruta de Python:", sys.prefix)

# Verificar que torch esté disponible
try:
    import torch
    print(f"✅ PyTorch {torch.__version__} encontrado")
    print(f"CUDA disponible: {torch.cuda.is_available()}")
except ImportError as e:
    print(f"❌ Error: {e}")

In [ ]:
# Celda 1 Imports básicos
import sys
import warnings
warnings.filterwarnings('ignore')

# Agregar src al path para poder importar nuestros módulos
sys.path.append('..')

print("✅ Imports básicos completados")

In [ ]:
# Celda 2 Imports del proyecto
import pyarrow as pa
print(f"PyArrow version: {pa.__version__}")

import datasets
print(f"Datasets version: {datasets.__version__}")
# Imports del proyecto
from src.config import setup_project, print_config_summary
from src.models import ModelLoader
from src.utils import DataLoader

print("✅ Módulos del proyecto importados")

In [ ]:
# Celda 3 - Imports para análisis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    precision_recall_fscore_support,
    confusion_matrix,
    roc_auc_score,
    classification_report,
    matthews_corrcoef
)

from tqdm.notebook import tqdm

# Configurar estilo de gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('viridis')

print("✅ Librerías de análisis importadas")

In [ ]:
# Celda 4 - Setup del proyecto
config = setup_project()

# Imprimir resumen
print_config_summary(config)

In [ ]:
# Celda 5 - Cargar modelo
print("🚀 Cargando modelo DistilBERT...\n")
model = ModelLoader(config)

# Información del modelo
print("\n📊 Información del Modelo:")
info = model.get_model_info()
for key, value in info.items():
    print(f"  • {key}: {value}")

In [ ]:
# Celda 5.1 Diagnóstico del modelo - Entrada/Salida
from src.utils.data_model import verificar_entrada_salida_modelo
print("🔬 DIAGNÓSTICO COMPLETO DE ENTRADA/SALIDA DEL MODELO")
print("="*60)

info_io = verificar_entrada_salida_modelo(model)

In [ ]:
# Celda 6 - Cargar dataset // devemos volver a usar el servicio de config
print("📥 CARGANDO DATASET SST-2")
print("="*60)

from datasets import load_dataset

# Cargar dataset completo sin procesamiento
print("\nCargando dataset SST-2 desde HuggingFace...")
dataset_raw = load_dataset("sst2")

print("\n✅ Dataset cargado exitosamente")
print(f"Splits disponibles: {list(dataset_raw.keys())}")

In [ ]:
# # Celda 6 - Cargar dataset
# print("📥 Cargando dataset IMDb...\n")
# dataset_raw = DataLoader(config)

# # Información del dataset
# print("\n📊 Información del Dataset:")
# dataset_info = dataset_raw.get_dataset_info()
# for key, value in dataset_info.items():
#     if isinstance(value, dict):
#         print(f"  • {key}:")
#         for k, v in value.items():
#             print(f"      - {k}: {v}")
#     else:
#         print(f"  • {key}: {value}")

In [ ]:
# Celda 6.1 - Análisis exploratorio del dataset SST-2
print("\n📊 ANÁLISIS EXPLORATORIO DEL DATASET SST-2")
print("="*60)
from src.utils.data_view import view_distributions
view = view_distributions(dataset_raw)

In [ ]:
from src.utils.data_model import diagnosticar_problema_completo
diagnostico = diagnosticar_problema_completo(model, dataset_raw['validation'])

In [ ]:
# # Celda 6.2 - Visualización de Distribuciones
# print("\n📈 VISUALIZACIÓN DE DISTRIBUCIONES")
# print("="*60)

# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np


# # Configurar estilo
# plt.style.use('seaborn-v0_8-darkgrid')
# sns.set_palette("husl")

# # Crear figura con subplots
# fig, axes = plt.subplots(2, 3, figsize=(15, 8))
# fig.suptitle('Análisis del Dataset SST-2', fontsize=16, fontweight='bold')

# # 1. Distribución de longitudes - Train
# ax1 = axes[0, 0]
# ax1.hist(train_lengths, bins=30, edgecolor='black', alpha=0.7)
# ax1.axvline(np.mean(train_lengths), color='red', linestyle='--', label=f'Media: {np.mean(train_lengths):.1f}')
# ax1.set_xlabel('Número de palabras')
# ax1.set_ylabel('Frecuencia')
# ax1.set_title('Distribución de Longitudes - Train')
# ax1.legend()

# # 2. Distribución de longitudes - Validation
# ax2 = axes[0, 1]
# ax2.hist(val_lengths, bins=30, edgecolor='black', alpha=0.7, color='orange')
# ax2.axvline(np.mean(val_lengths), color='red', linestyle='--', label=f'Media: {np.mean(val_lengths):.1f}')
# ax2.set_xlabel('Número de palabras')
# ax2.set_ylabel('Frecuencia')
# ax2.set_title('Distribución de Longitudes - Validation')
# ax2.legend()

# # 3. Balance de clases - Train
# ax3 = axes[0, 2]
# ax3.bar(['Negativo', 'Positivo'], [train_neg, train_pos], color=['#FF6B6B', '#4ECDC4'])
# ax3.set_ylabel('Cantidad')
# ax3.set_title('Balance de Clases - Train')
# for i, v in enumerate([train_neg, train_pos]):
#     ax3.text(i, v + 500, str(v), ha='center', fontweight='bold')

# # 4. Balance de clases - Validation
# ax4 = axes[1, 0]
# ax4.bar(['Negativo', 'Positivo'], [val_neg, val_pos], color=['#FF6B6B', '#4ECDC4'])
# ax4.set_ylabel('Cantidad')
# ax4.set_title('Balance de Clases - Validation')
# for i, v in enumerate([val_neg, val_pos]):
#     ax4.text(i, v + 10, str(v), ha='center', fontweight='bold')

# # 5. Boxplot de longitudes
# ax5 = axes[1, 1]
# ax5.boxplot([train_lengths, val_lengths], labels=['Train', 'Validation'])
# ax5.set_ylabel('Número de palabras')
# ax5.set_title('Comparación de Longitudes')

# # 6. Pie chart de balance general
# ax6 = axes[1, 2]
# total_neg = train_neg + val_neg
# total_pos = train_pos + val_pos
# ax6.pie([total_neg, total_pos], labels=['Negativo', 'Positivo'], 
#         autopct='%1.1f%%', colors=['#FF6B6B', '#4ECDC4'])
# ax6.set_title('Balance Global del Dataset')

# plt.tight_layout()
# plt.show()

# print("\n✅ Visualizaciones completadas")

In [ ]:
# ============================================================
# Celda 6.3 - VALIDACIÓN Y CORRECCIÓN AUTOMÁTICA DE LABELS
# ============================================================
from src.utils.data_format import validate_and_fix_labels

# Aplicar la validación y corrección al dataset cargado
dataset_raw = validate_and_fix_labels(dataset_raw)

print("\n📋 Dataset listo para usar con labels en formato correcto (0/1)")

In [ ]:
# ============================================================
# Celda 6.4 - PREPARACIÓN Y DIVISIÓN DE DATOS
# ============================================================
# Esta celda prepara los datos para la evaluación del modelo
# ============================================================

# Importar la función de preparación
from src.utils.data_preparation import preparar_datos_para_evaluacion

# Opción 1: Usar el split de validación completo
test_texts, test_labels = preparar_datos_para_evaluacion(
    dataset_raw, 
    split_test='validation',  # Puedes cambiar a 'test' si prefieres
    sample_size=None,         # None = usar todo el dataset
    random_seed=42
)

# Opción 2: Si quieres una muestra más pequeña para pruebas rápidas
# test_texts, test_labels = preparar_datos_para_evaluacion(
#     dataset_raw, 
#     split_test='validation',
#     sample_size=100,  # Solo 100 ejemplos para pruebas rápidas
#     random_seed=42
# )

print(f"\n✅ Datos listos para evaluación:")
print(f"  • Total de textos: {len(test_texts)}")
print(f"  • Total de etiquetas: {len(test_labels)}")
print(f"\n📝 Ejemplos de los datos preparados:")
for i in range(min(3, len(test_texts))):
    print(f"\n  Ejemplo {i+1}:")
    print(f"  • Texto: '{test_texts[i][:60]}...'")
    print(f"  • Etiqueta: {test_labels[i]} ({'Positive' if test_labels[i]==1 else 'Negative'})")

In [ ]:
# ============================================================
# Celda 7 - REALIZAR PREDICCIONES
# ============================================================
# Esta celda genera las predicciones usando el modelo cargado
# y aplica la corrección necesaria para el campo 'prediction'
# ============================================================

from src.utils.data_model import corregir_predicciones_modelo
import time

print("🚀 GENERACIÓN DE PREDICCIONES")
print("="*60)

# --------------------------------------------------------
# PASO 1: Realizar predicciones
# --------------------------------------------------------
print("\n1️⃣ Ejecutando predicciones en el conjunto de prueba...")
print("-"*40)

# Medir tiempo de inferencia
start_time = time.time()

# Hacer predicciones en batch (más eficiente)
print(f"  • Procesando {len(test_texts)} textos...")
predictions_raw = model.predict_batch(test_texts)

# Calcular tiempo transcurrido
elapsed_time = time.time() - start_time
avg_time = elapsed_time / len(test_texts)

print(f"  • ✅ Predicciones completadas")
print(f"  • ⏱️ Tiempo total: {elapsed_time:.2f} segundos")
print(f"  • ⏱️ Tiempo promedio por texto: {avg_time*1000:.2f} ms")

# --------------------------------------------------------
# PASO 2: Verificar estructura de predicciones
# --------------------------------------------------------
print("\n2️⃣ Verificación de estructura de predicciones...")
print("-"*40)

# Mostrar ejemplo de predicción raw
print("  Ejemplo de predicción completa:")
ejemplo = predictions_raw[0]
for key, value in ejemplo.items():
    if key != 'text':  # No mostrar el texto completo
        print(f"    • {key}: {value}")

# --------------------------------------------------------
# PASO 3: Corregir predicciones (usar label_id)
# --------------------------------------------------------
print("\n3️⃣ Aplicando corrección a las predicciones...")
print("-"*40)

# Usar la función que creamos para corregir
predicted_labels = corregir_predicciones_modelo(predictions_raw)

# --------------------------------------------------------
# PASO 4: Validación de integridad
# --------------------------------------------------------
print("\n4️⃣ Validación de integridad...")
print("-"*40)

# Verificar dimensiones
assert len(predicted_labels) == len(test_labels), "Error: Dimensiones no coinciden"
print(f"  • ✅ Dimensiones correctas: {len(predicted_labels)} predicciones")

# Verificar tipos de datos
assert all(isinstance(label, int) for label in predicted_labels[:10]), "Error: Labels deben ser enteros"
assert all(label in [0, 1] for label in predicted_labels[:10]), "Error: Labels deben ser 0 o 1"
print(f"  • ✅ Tipos de datos correctos")

# Mostrar distribución de predicciones
pred_neg = predicted_labels.count(0)
pred_pos = predicted_labels.count(1)
print(f"\n📊 Distribución de predicciones:")
print(f"  • Negativos (0): {pred_neg} ({pred_neg/len(predicted_labels)*100:.1f}%)")
print(f"  • Positivos (1): {pred_pos} ({pred_pos/len(predicted_labels)*100:.1f}%)")

# --------------------------------------------------------
# PASO 5: Guardar predicciones para análisis posterior
# --------------------------------------------------------
# Guardar las predicciones completas para análisis detallado
predictions_data = {
    'raw_predictions': predictions_raw,
    'corrected_labels': predicted_labels,
    'true_labels': test_labels,
    'texts': test_texts
}

print("\n" + "="*60)
print("✅ Predicciones generadas y corregidas exitosamente")
print(f"📦 Variables disponibles:")
print(f"  • predicted_labels: Lista de etiquetas predichas (0/1)")
print(f"  • predictions_raw: Predicciones completas con probabilidades")
print(f"  • predictions_data: Diccionario con todos los datos")

In [ ]:
# ============================================================
# Celda 8 - EVALUACIÓN COMPLETA DEL MODELO
# ============================================================
# Esta celda calcula todas las métricas de evaluación
# ============================================================

print("📊 EVALUACIÓN COMPLETA DEL MODELO")
print("="*60)

# --------------------------------------------------------
# MÉTRICAS BÁSICAS
# --------------------------------------------------------
print("\n🎯 MÉTRICAS PRINCIPALES:")
print("-"*40)

# Accuracy
accuracy = accuracy_score(test_labels, predicted_labels)
print(f"  • Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

# Precision, Recall, F1 por clase
precision, recall, f1, support = precision_recall_fscore_support(
    test_labels, predicted_labels, average=None
)

print(f"\n  Clase 0 (Negative):")
print(f"    • Precision: {precision[0]:.4f}")
print(f"    • Recall: {recall[0]:.4f}")
print(f"    • F1-Score: {f1[0]:.4f}")
print(f"    • Support: {support[0]}")

print(f"\n  Clase 1 (Positive):")
print(f"    • Precision: {precision[1]:.4f}")
print(f"    • Recall: {recall[1]:.4f}")
print(f"    • F1-Score: {f1[1]:.4f}")
print(f"    • Support: {support[1]}")

# Métricas promedio
avg_precision = np.mean(precision)
avg_recall = np.mean(recall)
avg_f1 = np.mean(f1)

print(f"\n  Promedios (Macro):")
print(f"    • Avg Precision: {avg_precision:.4f}")
print(f"    • Avg Recall: {avg_recall:.4f}")
print(f"    • Avg F1-Score: {avg_f1:.4f}")

# --------------------------------------------------------
# MÉTRICAS AVANZADAS
# --------------------------------------------------------
print("\n🔬 MÉTRICAS AVANZADAS:")
print("-"*40)

# Matthews Correlation Coefficient (bueno para clases desbalanceadas)
mcc = matthews_corrcoef(test_labels, predicted_labels)
print(f"  • Matthews Correlation Coefficient: {mcc:.4f}")

# ROC-AUC si tenemos probabilidades
if predictions_raw and 'probabilities' in predictions_raw[0]:
    # Extraer probabilidades de la clase positiva
    probs_positive = [pred['probabilities']['POSITIVE'] for pred in predictions_raw]
    roc_auc = roc_auc_score(test_labels, probs_positive)
    print(f"  • ROC-AUC Score: {roc_auc:.4f}")

# --------------------------------------------------------
# CLASSIFICATION REPORT DETALLADO
# --------------------------------------------------------
print("\n📋 CLASSIFICATION REPORT DETALLADO:")
print("-"*40)

report = classification_report(
    test_labels, 
    predicted_labels,
    target_names=['Negative', 'Positive'],
    digits=4
)
print(report)

# --------------------------------------------------------
# CONFUSION MATRIX
# --------------------------------------------------------
print("\n🔲 CONFUSION MATRIX:")
print("-"*40)

cm = confusion_matrix(test_labels, predicted_labels)
tn, fp, fn, tp = cm.ravel()

# Mostrar matriz formateada
print("\n                 Predicción")
print("                 Neg    Pos")
print(f"Real Negative: [{tn:4d}] [{fp:4d}]  → {tn+fp} total")
print(f"     Positive: [{fn:4d}] [{tp:4d}]  → {fn+tp} total")
print(f"                ----   ----")
print(f"               {tn+fn:5d}  {fp+tp:5d}")

# Tasas de error
print(f"\n📊 Análisis de Errores:")
print(f"  • False Positive Rate: {fp/(fp+tn)*100:.2f}%")
print(f"  • False Negative Rate: {fn/(fn+tp)*100:.2f}%")
print(f"  • Total de errores: {fp+fn} de {len(test_labels)} ({(fp+fn)/len(test_labels)*100:.2f}%)")

# --------------------------------------------------------
# INTERPRETACIÓN DE RESULTADOS
# --------------------------------------------------------
print("\n💡 INTERPRETACIÓN DE RESULTADOS:")
print("-"*40)

# Interpretar accuracy
if accuracy >= 0.95:
    nivel = "🏆 EXCELENTE"
    desc = "Rendimiento sobresaliente"
elif accuracy >= 0.90:
    nivel = "🎯 MUY BUENO"
    desc = "Rendimiento muy sólido"
elif accuracy >= 0.85:
    nivel = "✅ BUENO"
    desc = "Rendimiento satisfactorio"
elif accuracy >= 0.80:
    nivel = "👍 ACEPTABLE"
    desc = "Rendimiento decente"
else:
    nivel = "⚠️ MEJORABLE"
    desc = "Requiere optimización"

print(f"  Nivel de rendimiento: {nivel}")
print(f"  {desc} con {accuracy:.1%} de accuracy")

# Análisis de balance entre precision y recall
if abs(avg_precision - avg_recall) < 0.05:
    print(f"\n  ✅ Modelo balanceado: Precision y Recall similares")
elif avg_precision > avg_recall:
    print(f"\n  📌 Modelo conservador: Mayor precision que recall")
    print(f"     (Pocos falsos positivos, pero puede perder algunos positivos)")
else:
    print(f"\n  📌 Modelo agresivo: Mayor recall que precision")
    print(f"     (Captura muchos positivos, pero con más falsos positivos)")

print("\n" + "="*60)
print("✅ Evaluación completada exitosamente")

In [ ]:
# # Celda 8 - Predicciones CON TRUNCAMIENTO FORZADO 
# print("🔍 Realizando predicciones en el conjunto de test...\n")

# # Función para predecir con truncamiento
# def predict_with_truncation(model, texts):
#     """Predice con truncamiento forzado a 512 tokens"""
#     results = []
#     for text in texts:
#         # Tokenizar con truncamiento
#         inputs = model.tokenizer(
#             text, 
#             max_length=512,
#             truncation=True,
#             padding='max_length',
#             return_tensors='pt'
#         )
        
#         # Mover inputs a device
#         inputs = {k: v.to(model.device) for k, v in inputs.items()}
        
#         # Predecir
#         with torch.no_grad():
#             outputs = model.model(**inputs)
        
#         # Obtener probabilidades
#         probs = torch.softmax(outputs.logits, dim=-1)
#         conf, pred_class = torch.max(probs, dim=-1)
        
#         pred_label = model.model.config.id2label[pred_class.item()] 
        
#         results.append({
#             'text': text,
#             'predicted_label': pred_label, 
#             'predicted_class': pred_class.item(),
#             'confidence': conf.item(),
#             'probabilities': probs[0].tolist()
#         })
    
#     return results

# # Predecir en batches
# batch_size = config.model.batch_size
# all_preds = []

# for i in tqdm(range(0, len(test_texts), batch_size), desc="Prediciendo"):
#     batch_texts = test_texts[i : i+batch_size]
    
#     try:
#         # Intentar predecir normalmente primero
#         preds = model.predict_batch(batch_texts)
#         all_preds.extend(preds)
#     except RuntimeError:
#         print(f"⚠️ Error en batch {i//batch_size}, aplicando truncamiento...")
#         preds = predict_with_truncation(model, batch_texts)
#         all_preds.extend(preds)

# print(f"\n✅ Predicciones completadas: {len(all_preds)} ejemplos")

In [ ]:
# ============================================================
# Celda 9 - CREAR DATAFRAME CON RESULTADOS
# ============================================================
# Esta celda organiza todos los resultados en un DataFrame
# para facilitar el análisis y exportación
# ============================================================


print("📊 CREACIÓN DE DATAFRAME CON RESULTADOS")
print("="*60)

# --------------------------------------------------------
# PASO 1: Crear DataFrame principal
# --------------------------------------------------------
print("\n1️⃣ Construyendo DataFrame con predicciones...")
print("-"*40)

# Crear DataFrame con todos los datos
results_df = pd.DataFrame({
    'text': test_texts,
    'true_label': test_labels,
    'predicted_label': predicted_labels,
    'true_sentiment': ['Positive' if label == 1 else 'Negative' for label in test_labels],
    'predicted_sentiment': ['Positive' if label == 1 else 'Negative' for label in predicted_labels]
})

# Agregar probabilidades
results_df['prob_negative'] = [pred['probabilities']['NEGATIVE'] for pred in predictions_raw]
results_df['prob_positive'] = [pred['probabilities']['POSITIVE'] for pred in predictions_raw]

# Agregar confianza del modelo (máxima probabilidad)
results_df['confidence'] = results_df[['prob_negative', 'prob_positive']].max(axis=1)

# Marcar si la predicción fue correcta
results_df['correct'] = results_df['true_label'] == results_df['predicted_label']

# Categorizar tipo de error
def categorize_error(row):
    if row['correct']:
        return 'Correct'
    elif row['true_label'] == 0 and row['predicted_label'] == 1:
        return 'False Positive'
    elif row['true_label'] == 1 and row['predicted_label'] == 0:
        return 'False Negative'
    return 'Unknown'

results_df['error_type'] = results_df.apply(categorize_error, axis=1)

print(f"✅ DataFrame creado con {len(results_df)} filas y {len(results_df.columns)} columnas")

# --------------------------------------------------------
# PASO 2: Mostrar información del DataFrame
# --------------------------------------------------------
print("\n2️⃣ Información del DataFrame:")
print("-"*40)

print("\nColumnas disponibles:")
for col in results_df.columns:
    print(f"  • {col}: {results_df[col].dtype}")

print(f"\nPrimeras 5 filas:")
print(results_df[['text', 'true_sentiment', 'predicted_sentiment', 'confidence', 'correct']].head())

# --------------------------------------------------------
# PASO 3: Estadísticas rápidas
# --------------------------------------------------------
print("\n3️⃣ Estadísticas del DataFrame:")
print("-"*40)

# Distribución de predicciones
print("\n📊 Distribución de resultados:")
print(results_df['error_type'].value_counts())
print("\n📊 Porcentajes:")
print(results_df['error_type'].value_counts(normalize=True).round(4) * 100)

# Confianza promedio por tipo
print("\n📊 Confianza promedio por tipo de resultado:")
for error_type in results_df['error_type'].unique():
    avg_conf = results_df[results_df['error_type'] == error_type]['confidence'].mean()
    print(f"  • {error_type}: {avg_conf:.4f}")

# --------------------------------------------------------
# PASO 4: Identificar casos interesantes
# --------------------------------------------------------
print("\n4️⃣ Casos interesantes para análisis:")
print("-"*40)

# Predicciones incorrectas con alta confianza
high_conf_errors = results_df[(~results_df['correct']) & (results_df['confidence'] > 0.9)]
print(f"\n⚠️ Errores con alta confianza (>90%): {len(high_conf_errors)} casos")
if len(high_conf_errors) > 0:
    print("\nEjemplos de errores con alta confianza:")
    for idx in high_conf_errors.head(3).index:
        row = results_df.loc[idx]
        print(f"\n  • Texto: '{row['text'][:60]}...'")
        print(f"    Real: {row['true_sentiment']}, Predicho: {row['predicted_sentiment']}")
        print(f"    Confianza: {row['confidence']:.2%}")

# Predicciones correctas con baja confianza
low_conf_correct = results_df[(results_df['correct']) & (results_df['confidence'] < 0.6)]
print(f"\n📌 Aciertos con baja confianza (<60%): {len(low_conf_correct)} casos")

# --------------------------------------------------------
# PASO 5: Preparar DataFrames específicos
# --------------------------------------------------------
print("\n5️⃣ DataFrames específicos creados:")
print("-"*40)

# DataFrame solo con errores
errors_df = results_df[~results_df['correct']].copy()
print(f"  • errors_df: {len(errors_df)} errores para análisis detallado")

# DataFrame con casos extremos
extreme_cases_df = results_df[
    (results_df['confidence'] > 0.95) | (results_df['confidence'] < 0.55)
].copy()
print(f"  • extreme_cases_df: {len(extreme_cases_df)} casos con confianza muy alta o muy baja")

# DataFrame ordenado por confianza
sorted_df = results_df.sort_values('confidence', ascending=False).copy()
print(f"  • sorted_df: DataFrame ordenado por confianza")

print("\n" + "="*60)
print("✅ DataFrames creados exitosamente")
print("\n📦 Variables disponibles:")
print("  • results_df: DataFrame principal con todos los resultados")
print("  • errors_df: Solo las predicciones incorrectas")
print("  • extreme_cases_df: Casos con confianza extrema")
print("  • sorted_df: Resultados ordenados por confianza")

In [ ]:
# # Celda 9 - Crear DataFrame
# predicted_labels = [pred['label_id'] for pred in all_preds]
# predicted_probs = [pred['confidence'] for pred in all_preds]
# predicted_classes = [pred['prediction'] for pred in all_preds]

# # Crear DataFrame con resultados
# results_df = pd.DataFrame({
#     'text': test_texts,
#     'true_label': test_labels,
#     'predicted_label': predicted_labels,
#     'prediction': predicted_classes,
#     'confidence': predicted_probs,
#     'correct': [t == p for t, p in zip(test_labels, predicted_labels)]
# })

# # Mostrar primeros ejemplos
# print("\n📊 Primeros 5 resultados:")
# display(results_df[['text', 'true_label', 'prediction', 'confidence', 'correct']].head())

In [ ]:
# Celda 10 - Métricas
accuracy = accuracy_score(test_labels, predicted_labels)
precision = precision_score(test_labels, predicted_labels)
recall = recall_score(test_labels, predicted_labels)
f1 = f1_score(test_labels, predicted_labels)

print("📈 MÉTRICAS DE CLASIFICACIÓN")
print("="*50)
print(f"Accuracy:  {accuracy:.4f} ({accuracy:.2%})")
print(f"Precision: {precision:.4f} ({precision:.2%})")
print(f"Recall:    {recall:.4f} ({recall:.2%})")
print(f"F1-Score:  {f1:.4f} ({f1:.2%})")
print("="*50)

# Reporte completo
print("\n📋 Reporte de Clasificación:")
print(classification_report(
    test_labels, 
    predicted_labels,
    target_names=['NEGATIVE', 'POSITIVE']
))

Predicción
              NEG    POS
Real  NEG  [  TN  |  FP  ]
      POS  [  FN  |  TP  ]

In [ ]:
# ============================================================
# Celda 11 - VISUALIZACIONES DE RESULTADOS
# ============================================================
# Esta celda crea visualizaciones para analizar el rendimiento
# ============================================================

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc

# Configurar estilo de visualización
plt.style.use('default')
sns.set_palette("husl")

print("📊 VISUALIZACIONES DE RESULTADOS")
print("="*60)

# Crear figura con subplots
fig = plt.figure(figsize=(16, 10))

# --------------------------------------------------------
# 1. MATRIZ DE CONFUSIÓN
# --------------------------------------------------------
ax1 = plt.subplot(2, 3, 1)
cm = confusion_matrix(test_labels, predicted_labels)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Negative', 'Positive'],
            yticklabels=['Negative', 'Positive'],
            cbar_kws={'label': 'Cantidad'})
plt.title('Matriz de Confusión', fontsize=14, fontweight='bold')
plt.ylabel('Etiqueta Real')
plt.xlabel('Predicción')

# Agregar porcentajes a la matriz
for i in range(2):
    for j in range(2):
        percentage = cm[i, j] / cm.sum() * 100
        plt.text(j + 0.5, i + 0.7, f'({percentage:.1f}%)', 
                ha='center', va='center', fontsize=9, color='gray')

# --------------------------------------------------------
# 2. DISTRIBUCIÓN DE PROBABILIDADES
# --------------------------------------------------------
ax2 = plt.subplot(2, 3, 2)

# Separar probabilidades por clase real
neg_probs = [predictions_raw[i]['probabilities']['POSITIVE'] 
             for i in range(len(test_labels)) if test_labels[i] == 0]
pos_probs = [predictions_raw[i]['probabilities']['POSITIVE'] 
             for i in range(len(test_labels)) if test_labels[i] == 1]

# Crear histograma
bins = np.linspace(0, 1, 31)
ax2.hist(neg_probs, bins=bins, alpha=0.5, label='Real Negative', color='red', edgecolor='black')
ax2.hist(pos_probs, bins=bins, alpha=0.5, label='Real Positive', color='green', edgecolor='black')
ax2.axvline(x=0.5, color='black', linestyle='--', alpha=0.5, label='Threshold')
ax2.set_xlabel('Probabilidad de Positivo')
ax2.set_ylabel('Frecuencia')
ax2.set_title('Distribución de Probabilidades', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# --------------------------------------------------------
# 3. CURVA ROC
# --------------------------------------------------------
ax3 = plt.subplot(2, 3, 3)

# Calcular curva ROC
probs_positive = [pred['probabilities']['POSITIVE'] for pred in predictions_raw]
fpr, tpr, thresholds = roc_curve(test_labels, probs_positive)
roc_auc = auc(fpr, tpr)

# Plotear
ax3.plot(fpr, tpr, color='darkorange', lw=2, 
         label=f'ROC curve (AUC = {roc_auc:.3f})')
ax3.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
ax3.fill_between(fpr, tpr, alpha=0.2, color='darkorange')
ax3.set_xlim([0.0, 1.0])
ax3.set_ylim([0.0, 1.05])
ax3.set_xlabel('False Positive Rate')
ax3.set_ylabel('True Positive Rate')
ax3.set_title('Curva ROC', fontsize=14, fontweight='bold')
ax3.legend(loc="lower right")
ax3.grid(True, alpha=0.3)

# --------------------------------------------------------
# 4. MÉTRICAS POR CLASE
# --------------------------------------------------------
ax4 = plt.subplot(2, 3, 4)

# Calcular métricas para esta celda
from sklearn.metrics import precision_recall_fscore_support
precision, recall, f1, support = precision_recall_fscore_support(
    test_labels, predicted_labels, average=None
)

# Datos de métricas
metrics_data = {
    'Precision': [precision[0], precision[1]],
    'Recall': [recall[0], recall[1]],
    'F1-Score': [f1[0], f1[1]]
}

x = np.arange(2)
width = 0.25

# Crear barras
for i, (metric, values) in enumerate(metrics_data.items()):
    offset = (i - 1) * width
    bars = ax4.bar(x + offset, values, width, label=metric)
    
    # Agregar valores encima de las barras
    for bar, val in zip(bars, values):
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:.3f}', ha='center', va='bottom', fontsize=9)

ax4.set_xlabel('Clase')
ax4.set_ylabel('Score')
ax4.set_title('Métricas por Clase', fontsize=14, fontweight='bold')
ax4.set_xticks(x)
ax4.set_xticklabels(['Negative', 'Positive'])
ax4.legend()
ax4.set_ylim([0, 1.1])
ax4.grid(True, alpha=0.3, axis='y')

# --------------------------------------------------------
# 5. DISTRIBUCIÓN DE CONFIANZA
# --------------------------------------------------------
ax5 = plt.subplot(2, 3, 5)

# Calcular confianza para cada predicción
confidences = results_df['confidence'].values

# Separar por correctas e incorrectas
correct_conf = results_df[results_df['correct']]['confidence'].values
incorrect_conf = results_df[~results_df['correct']]['confidence'].values

# Crear boxplot
bp = ax5.boxplot([correct_conf, incorrect_conf], 
                  labels=['Correctas', 'Incorrectas'],
                  patch_artist=True,
                  showmeans=True)

# Colorear las cajas
colors = ['lightgreen', 'lightcoral']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax5.set_ylabel('Confianza')
ax5.set_title('Distribución de Confianza', fontsize=14, fontweight='bold')
ax5.grid(True, alpha=0.3, axis='y')

# Agregar estadísticas
ax5.text(1, 0.55, f'μ={correct_conf.mean():.3f}', ha='center', fontsize=9)
ax5.text(2, 0.55, f'μ={incorrect_conf.mean():.3f}', ha='center', fontsize=9)

# --------------------------------------------------------
# 6. ANÁLISIS DE ERRORES
# --------------------------------------------------------
ax6 = plt.subplot(2, 3, 6)

# Contar tipos de errores
error_counts = results_df['error_type'].value_counts()

# Crear gráfico de torta
colors_pie = ['#90EE90', '#FFB6C1', '#FFA07A']
wedges, texts, autotexts = ax6.pie(error_counts.values, 
                                     labels=error_counts.index,
                                     autopct='%1.1f%%',
                                     colors=colors_pie,
                                     startangle=90,
                                     explode=(0, 0.1, 0.1))

# Mejorar apariencia
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')
    autotext.set_fontsize(10)

ax6.set_title('Distribución de Predicciones', fontsize=14, fontweight='bold')

# Agregar leyenda con cantidades
legend_labels = [f'{label}: {count}' for label, count in error_counts.items()]
ax6.legend(legend_labels, loc='upper left', bbox_to_anchor=(1, 1))

# Ajustar layout
plt.suptitle(f'Análisis de Resultados - Accuracy: {accuracy:.2%}', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()

# Mostrar el gráfico
plt.show()

# --------------------------------------------------------
# ESTADÍSTICAS ADICIONALES
# --------------------------------------------------------
print("\n📊 ESTADÍSTICAS ADICIONALES:")
print("-"*60)

# Análisis de errores
print("\n🔍 Análisis detallado de errores:")
false_positives = len(results_df[results_df['error_type'] == 'False Positive'])
false_negatives = len(results_df[results_df['error_type'] == 'False Negative'])
total_errors = false_positives + false_negatives

print(f"  • False Positives: {false_positives} ({false_positives/len(results_df)*100:.2f}%)")
print(f"  • False Negatives: {false_negatives} ({false_negatives/len(results_df)*100:.2f}%)")
print(f"  • Total de errores: {total_errors} ({total_errors/len(results_df)*100:.2f}%)")

# Confianza promedio
print(f"\n📈 Confianza del modelo:")
print(f"  • Confianza promedio global: {results_df['confidence'].mean():.4f}")
print(f"  • Confianza en predicciones correctas: {correct_conf.mean():.4f}")
print(f"  • Confianza en predicciones incorrectas: {incorrect_conf.mean():.4f}")
print(f"  • Diferencia: {correct_conf.mean() - incorrect_conf.mean():.4f}")

# Casos extremos
very_confident_errors = results_df[(~results_df['correct']) & (results_df['confidence'] > 0.95)]
print(f"\n⚠️ Errores con muy alta confianza (>95%):")
print(f"  • Cantidad: {len(very_confident_errors)} casos")
if len(very_confident_errors) > 0:
    print(f"  • Porcentaje del total de errores: {len(very_confident_errors)/total_errors*100:.1f}%")

print("\n" + "="*60)
print("✅ Visualizaciones completadas exitosamente")

In [ ]:
# Celda 11 - Matriz de Confusión
# Para mi objetivo esta visualización es clave
# Calcular matriz de confusión
cm = confusion_matrix(test_labels, predicted_labels)

# Visualizar
plt.figure(figsize=(8, 6))
sns.heatmap(
    cm, 
    annot=True,           # Mostrar números
    fmt='d',              # Formato entero
    cmap='Blues',         # Colores azules
    xticklabels=['NEGATIVE', 'POSITIVE'],
    yticklabels=['NEGATIVE', 'POSITIVE'],
    cbar_kws={'label': 'Cantidad'}
)
plt.title('Matriz de Confusión', fontsize=14, fontweight='bold')
plt.ylabel('Etiqueta Real', fontsize=12)
plt.xlabel('Predicción', fontsize=12)
plt.tight_layout()
plt.show()

# Análisis de errores
tn, fp, fn, tp = cm.ravel()
print("\n📊 Análisis de la Matriz:")
print(f"  • True Negatives (TN):  {tn} - Negativos correctos")
print(f"  • False Positives (FP): {fp} - Negativos clasificados como positivos")
print(f"  • False Negatives (FN): {fn} - Positivos clasificados como negativos")
print(f"  • True Positives (TP):  {tp} - Positivos correctos")
print(f"\n  • Total de errores: {fp + fn} ({(fp + fn)/len(test_labels):.2%})")

In [ ]:
# ============================================================
# Celda 12 - DIAGNÓSTICO PARA INTERPRETABILIDAD
# ============================================================
# Esta celda identifica qué herramientas tenemos disponibles
# para interpretar las decisiones del modelo
# ============================================================

print("🔍 DIAGNÓSTICO PASO A PASO - IDENTIFICAR QUÉ TENEMOS")
print("="*60)

# --------------------------------------------------------
# PASO 1: VERIFICAR QUÉ TIPO DE MODELO TENEMOS
# --------------------------------------------------------
print("\n1️⃣ TIPO DE MODELO Y ARQUITECTURA:")
print("-"*40)

# Información básica del modelo
model_info = model.get_model_info()
print(f"  • Nombre del modelo: {model_info.get('model_name', 'No especificado')}")
print(f"  • Tipo: Transformer (DistilBERT)")
print(f"  • Tarea: Clasificación de sentimientos binaria")
print(f"  • Clases: {model_info.get('id2label', 'No disponible')}")

# Verificar si tenemos acceso al modelo de HuggingFace
if hasattr(model, 'pipeline'):
    print(f"  • ✅ Pipeline de HuggingFace disponible")
    
    # Verificar componentes del pipeline
    if hasattr(model.pipeline, 'model'):
        print(f"  • ✅ Acceso al modelo base")
        model_base = model.pipeline.model
        print(f"      - Tipo: {type(model_base).__name__}")
        
    if hasattr(model.pipeline, 'tokenizer'):
        print(f"  • ✅ Acceso al tokenizer")
        tokenizer = model.pipeline.tokenizer
        print(f"      - Tipo: {type(tokenizer).__name__}")
else:
    print(f"  • ⚠️ Pipeline no directamente accesible")

# --------------------------------------------------------
# PASO 2: DATOS DISPONIBLES PARA INTERPRETACIÓN
# --------------------------------------------------------
print("\n2️⃣ DATOS DISPONIBLES PARA ANÁLISIS:")
print("-"*40)

# Verificar qué tenemos en memoria
available_data = {
    'test_texts': 'Textos de entrada' if 'test_texts' in locals() else None,
    'test_labels': 'Etiquetas reales' if 'test_labels' in locals() else None,
    'predicted_labels': 'Predicciones' if 'predicted_labels' in locals() else None,
    'predictions_raw': 'Predicciones completas' if 'predictions_raw' in locals() else None,
    'results_df': 'DataFrame de resultados' if 'results_df' in locals() else None,
    'errors_df': 'DataFrame de errores' if 'errors_df' in locals() else None
}

for var_name, description in available_data.items():
    if description:
        if var_name in locals():
            var = eval(var_name)
            if hasattr(var, '__len__'):
                print(f"  • ✅ {var_name}: {description} ({len(var)} elementos)")
            else:
                print(f"  • ✅ {var_name}: {description}")
    else:
        print(f"  • ❌ {var_name}: No disponible")

# --------------------------------------------------------
# PASO 3: INFORMACIÓN EN LAS PREDICCIONES
# --------------------------------------------------------
print("\n3️⃣ INFORMACIÓN DISPONIBLE EN PREDICCIONES:")
print("-"*40)

if 'predictions_raw' in locals() and len(predictions_raw) > 0:
    sample_pred = predictions_raw[0]
    print("  Campos en cada predicción:")
    for key in sample_pred.keys():
        print(f"    • {key}: {type(sample_pred[key]).__name__}")
    
    # Verificar si tenemos probabilidades por clase
    if 'probabilities' in sample_pred:
        print(f"\n  📊 Probabilidades disponibles para:")
        for class_name in sample_pred['probabilities'].keys():
            print(f"    • {class_name}")
        print("  ✅ Podemos analizar la confianza del modelo")
    
    # Verificar si tenemos scores de atención o tokens
    if 'attention_scores' in sample_pred:
        print("  ✅ Scores de atención disponibles")
    else:
        print("  ❌ Scores de atención no disponibles en predicciones")

# --------------------------------------------------------
# PASO 4: CAPACIDADES DE INTERPRETABILIDAD
# --------------------------------------------------------
print("\n4️⃣ TÉCNICAS DE INTERPRETABILIDAD APLICABLES:")
print("-"*40)

print("\n📊 Con los datos actuales podemos hacer:")
print("  ✅ Análisis de confianza y probabilidades")
print("  ✅ Análisis de errores por patrones")
print("  ✅ Análisis de casos extremos (alta/baja confianza)")
print("  ✅ Análisis estadístico de predicciones")

print("\n🔧 Para interpretabilidad avanzada necesitaríamos:")
print("  • LIME: Explicaciones locales (requiere instalar lime)")
print("  • SHAP: Valores de Shapley (requiere instalar shap)")
print("  • Attention Weights: Pesos de atención del transformer")
print("  • Integrated Gradients: Gradientes del modelo")

# --------------------------------------------------------
# PASO 5: ANÁLISIS BÁSICO DE INTERPRETABILIDAD
# --------------------------------------------------------
print("\n5️⃣ ANÁLISIS BÁSICO DE INTERPRETABILIDAD:")
print("-"*40)

# Analizar casos más seguros vs más inciertos
if 'results_df' in locals():
    # Casos más seguros (alta confianza y correctos)
    most_confident = results_df[results_df['correct']].nlargest(5, 'confidence')
    print("\n🎯 Predicciones más confiables (correctas con alta confianza):")
    for idx in most_confident.index[:3]:
        row = results_df.loc[idx]
        print(f"\n  • Texto: '{row['text'][:60]}...'")
        print(f"    Sentimiento: {row['true_sentiment']} (confianza: {row['confidence']:.2%})")
    
    # Casos más inciertos
    most_uncertain = results_df.nsmallest(5, 'confidence')
    print("\n🤔 Predicciones más inciertas (baja confianza):")
    for idx in most_uncertain.index[:3]:
        row = results_df.loc[idx]
        print(f"\n  • Texto: '{row['text'][:60]}...'")
        print(f"    Predicción: {row['predicted_sentiment']} (confianza: {row['confidence']:.2%})")
        print(f"    Prob Neg: {row['prob_negative']:.3f}, Prob Pos: {row['prob_positive']:.3f}")

# --------------------------------------------------------
# PASO 6: PREPARACIÓN PARA INTERPRETABILIDAD AVANZADA
# --------------------------------------------------------
print("\n6️⃣ PREPARACIÓN PARA INTERPRETABILIDAD AVANZADA:")
print("-"*40)

# Verificar qué librerías están disponibles
libraries_check = {
    'lime': False,
    'shap': False,
    'captum': False,
    'transformers_interpret': False
}

for lib_name in libraries_check.keys():
    try:
        __import__(lib_name)
        libraries_check[lib_name] = True
        print(f"  ✅ {lib_name}: Instalado y disponible")
    except ImportError:
        print(f"  ❌ {lib_name}: No instalado")

# Sugerencias de instalación
if not any(libraries_check.values()):
    print("\n💡 Para análisis de interpretabilidad avanzado, instala:")
    print("  pip install lime               # Para explicaciones locales")
    print("  pip install shap               # Para valores SHAP")
    print("  pip install captum             # Para PyTorch (si aplica)")
    print("  pip install transformers-interpret  # Para transformers")

# --------------------------------------------------------
# RESUMEN Y PRÓXIMOS PASOS
# --------------------------------------------------------
print("\n" + "="*60)
print("📋 RESUMEN Y PRÓXIMOS PASOS:")
print("-"*40)

print("\n✅ PODEMOS HACER AHORA:")
print("  1. Análisis de confianza y distribución de probabilidades")
print("  2. Identificación de patrones en errores")
print("  3. Análisis de casos extremos")
print("  4. Visualización de métricas por subgrupos")

print("\n🔄 PRÓXIMOS PASOS SUGERIDOS:")
print("  1. Analizar palabras clave que influyen en predicciones")
print("  2. Estudiar casos donde el modelo falla consistentemente")
print("  3. Implementar LIME para explicaciones locales")
print("  4. Extraer attention weights si es posible")

print("\n💾 VARIABLES LISTAS PARA INTERPRETABILIDAD:")
print(f"  • results_df: DataFrame con todos los resultados")
print(f"  • errors_df: Solo los errores para análisis profundo")
print(f"  • predictions_raw: Predicciones con probabilidades completas")

print("\n" + "="*60)
print("✅ Diagnóstico completado - Listo para análisis de interpretabilidad")

In [ ]:
# ============================================================
# Celda 13 - DISTRIBUCIÓN DE CONFIANZA
# ============================================================
# Esta celda analiza en detalle la distribución de confianza
# del modelo en sus predicciones
# ============================================================


print("📊 ANÁLISIS DE DISTRIBUCIÓN DE CONFIANZA")
print("="*60)

# --------------------------------------------------------
# PASO 1: ESTADÍSTICAS DE CONFIANZA
# --------------------------------------------------------
print("\n1️⃣ ESTADÍSTICAS GENERALES DE CONFIANZA:")
print("-"*40)

# Estadísticas básicas
confidence_stats = results_df['confidence'].describe()
print("\n📈 Resumen estadístico de confianza:")
print(confidence_stats)

# Estadísticas por tipo de resultado
print("\n📊 Confianza por tipo de resultado:")
for error_type in results_df['error_type'].unique():
    conf_values = results_df[results_df['error_type'] == error_type]['confidence']
    print(f"\n  {error_type}:")
    print(f"    • Media: {conf_values.mean():.4f}")
    print(f"    • Mediana: {conf_values.median():.4f}")
    print(f"    • Std: {conf_values.std():.4f}")
    print(f"    • Min: {conf_values.min():.4f}")
    print(f"    • Max: {conf_values.max():.4f}")
    print(f"    • Cantidad: {len(conf_values)} ({len(conf_values)/len(results_df)*100:.1f}%)")

# --------------------------------------------------------
# PASO 2: ANÁLISIS POR RANGOS DE CONFIANZA
# --------------------------------------------------------
print("\n2️⃣ ANÁLISIS POR RANGOS DE CONFIANZA:")
print("-"*40)

# Definir rangos de confianza
confidence_ranges = [
    (0.5, 0.6, 'Muy baja (50-60%)'),
    (0.6, 0.7, 'Baja (60-70%)'),
    (0.7, 0.8, 'Media (70-80%)'),
    (0.8, 0.9, 'Alta (80-90%)'),
    (0.9, 1.0, 'Muy alta (90-100%)')
]

print("\n📊 Distribución por rangos:")
for min_conf, max_conf, label in confidence_ranges:
    mask = (results_df['confidence'] >= min_conf) & (results_df['confidence'] < max_conf)
    count = mask.sum()
    correct_in_range = (results_df[mask]['correct']).sum()
    accuracy_in_range = correct_in_range / count if count > 0 else 0
    
    print(f"\n  {label}:")
    print(f"    • Cantidad: {count} ({count/len(results_df)*100:.1f}%)")
    if count > 0:
        print(f"    • Correctas: {correct_in_range} ({accuracy_in_range*100:.1f}%)")
        print(f"    • Incorrectas: {count - correct_in_range} ({(1-accuracy_in_range)*100:.1f}%)")

# --------------------------------------------------------
# PASO 3: CALIBRACIÓN DEL MODELO
# --------------------------------------------------------
print("\n3️⃣ CALIBRACIÓN DEL MODELO:")
print("-"*40)

# Analizar si la confianza refleja la probabilidad real de acierto
calibration_bins = [(0.5, 0.6), (0.6, 0.7), (0.7, 0.8), (0.8, 0.9), (0.9, 1.0)]
expected_acc = []
actual_acc = []
bin_centers = []

for min_conf, max_conf in calibration_bins:
    mask = (results_df['confidence'] >= min_conf) & (results_df['confidence'] < max_conf)
    if mask.sum() > 0:
        # Confianza esperada (centro del bin)
        expected = (min_conf + max_conf) / 2
        # Accuracy real en ese rango
        actual = results_df[mask]['correct'].mean()
        
        expected_acc.append(expected)
        actual_acc.append(actual)
        bin_centers.append(expected)
        
        print(f"  Confianza {min_conf:.0%}-{max_conf:.0%}:")
        print(f"    • Accuracy esperado: {expected:.1%}")
        print(f"    • Accuracy real: {actual:.1%}")
        print(f"    • Diferencia: {(actual - expected)*100:+.1f}%")

# --------------------------------------------------------
# PASO 4: VISUALIZACIONES
# --------------------------------------------------------
print("\n4️⃣ GENERANDO VISUALIZACIONES...")
print("-"*40)

# Crear figura con subplots
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
fig.suptitle('Análisis Detallado de Distribución de Confianza', fontsize=16, fontweight='bold')

# --------------------------------------------------------
# Subplot 1: Histograma de confianza general
# --------------------------------------------------------
ax1 = axes[0, 0]
ax1.hist(results_df['confidence'], bins=30, edgecolor='black', alpha=0.7, color='skyblue')
ax1.axvline(results_df['confidence'].mean(), color='red', linestyle='--', 
            label=f'Media: {results_df["confidence"].mean():.3f}')
ax1.axvline(results_df['confidence'].median(), color='green', linestyle='--', 
            label=f'Mediana: {results_df["confidence"].median():.3f}')
ax1.set_xlabel('Confianza')
ax1.set_ylabel('Frecuencia')
ax1.set_title('Distribución General de Confianza')
ax1.legend()
ax1.grid(True, alpha=0.3)

# --------------------------------------------------------
# Subplot 2: Distribución por tipo de resultado
# --------------------------------------------------------
ax2 = axes[0, 1]
correct_conf = results_df[results_df['correct']]['confidence']
incorrect_conf = results_df[~results_df['correct']]['confidence']

ax2.hist([correct_conf, incorrect_conf], bins=20, label=['Correctas', 'Incorrectas'], 
         color=['green', 'red'], alpha=0.6, edgecolor='black')
ax2.set_xlabel('Confianza')
ax2.set_ylabel('Frecuencia')
ax2.set_title('Confianza: Correctas vs Incorrectas')
ax2.legend()
ax2.grid(True, alpha=0.3)

# --------------------------------------------------------
# Subplot 3: Curva de calibración
# --------------------------------------------------------
ax3 = axes[0, 2]
if len(expected_acc) > 0:
    ax3.plot(expected_acc, actual_acc, 'o-', markersize=8, linewidth=2, label='Calibración real')
    ax3.plot([0.5, 1], [0.5, 1], 'k--', alpha=0.5, label='Perfectamente calibrado')
    
    # Área sombreada para mostrar desviación
    ax3.fill_between([0.5, 1], [0.45, 0.95], [0.55, 1.05], 
                     alpha=0.2, color='gray', label='±5% margen')
    
    ax3.set_xlabel('Confianza Esperada')
    ax3.set_ylabel('Accuracy Real')
    ax3.set_title('Curva de Calibración del Modelo')
    ax3.set_xlim([0.5, 1.0])
    ax3.set_ylim([0.5, 1.0])
    ax3.legend()
    ax3.grid(True, alpha=0.3)

# --------------------------------------------------------
# Subplot 4: Violin plot por sentimiento
# --------------------------------------------------------
ax4 = axes[1, 0]
data_for_violin = []
labels_for_violin = []

for sentiment in ['Negative', 'Positive']:
    for pred_type in ['Correct', 'Incorrect']:
        mask = (results_df['true_sentiment'] == sentiment) & \
               (results_df['correct'] == (pred_type == 'Correct'))
        if mask.sum() > 0:
            data_for_violin.append(results_df[mask]['confidence'].values)
            labels_for_violin.append(f'{sentiment}\n{pred_type}')

if data_for_violin:
    parts = ax4.violinplot(data_for_violin, positions=range(len(data_for_violin)), 
                           widths=0.7, showmeans=True, showmedians=True)
    ax4.set_xticks(range(len(labels_for_violin)))
    ax4.set_xticklabels(labels_for_violin, rotation=45)
    ax4.set_ylabel('Confianza')
    ax4.set_title('Distribución de Confianza por Sentimiento y Resultado')
    ax4.grid(True, alpha=0.3, axis='y')

# --------------------------------------------------------
# Subplot 5: Densidad de probabilidades
# --------------------------------------------------------
ax5 = axes[1, 1]

# KDE plot para distribuciones suaves
if len(correct_conf) > 1:
    correct_conf.plot.kde(ax=ax5, label='Predicciones Correctas', color='green', linewidth=2)
if len(incorrect_conf) > 1:
    incorrect_conf.plot.kde(ax=ax5, label='Predicciones Incorrectas', color='red', linewidth=2)

ax5.set_xlabel('Confianza')
ax5.set_ylabel('Densidad')
ax5.set_title('Densidad de Probabilidad de Confianza')
ax5.set_xlim([0.5, 1.0])
ax5.legend()
ax5.grid(True, alpha=0.3)

# --------------------------------------------------------
# Subplot 6: Matriz de confusión por nivel de confianza
# --------------------------------------------------------
ax6 = axes[1, 2]

# Crear matriz de confusión para diferentes niveles de confianza
conf_levels = ['Baja\n(50-70%)', 'Media\n(70-90%)', 'Alta\n(90-100%)']
conf_matrix_data = []

for level in [(0.5, 0.7), (0.7, 0.9), (0.9, 1.0)]:
    mask = (results_df['confidence'] >= level[0]) & (results_df['confidence'] < level[1])
    if mask.sum() > 0:
        tn = ((results_df[mask]['true_label'] == 0) & (results_df[mask]['predicted_label'] == 0)).sum()
        fp = ((results_df[mask]['true_label'] == 0) & (results_df[mask]['predicted_label'] == 1)).sum()
        fn = ((results_df[mask]['true_label'] == 1) & (results_df[mask]['predicted_label'] == 0)).sum()
        tp = ((results_df[mask]['true_label'] == 1) & (results_df[mask]['predicted_label'] == 1)).sum()
        total = mask.sum()
        conf_matrix_data.append([tn/total*100, fp/total*100, fn/total*100, tp/total*100])
    else:
        conf_matrix_data.append([0, 0, 0, 0])

# Visualizar como barras apiladas
x = np.arange(len(conf_levels))
width = 0.35
colors = ['#2E7D32', '#C62828', '#F57C00', '#1976D2']  # Verde, Rojo, Naranja, Azul
labels = ['TN', 'FP', 'FN', 'TP']

bottom = np.zeros(len(conf_levels))
for i, label in enumerate(labels):
    values = [row[i] for row in conf_matrix_data]
    ax6.bar(x, values, width*2, bottom=bottom, label=label, color=colors[i])
    bottom += values

ax6.set_ylabel('Porcentaje (%)')
ax6.set_xlabel('Nivel de Confianza')
ax6.set_title('Distribución de Predicciones por Confianza')
ax6.set_xticks(x)
ax6.set_xticklabels(conf_levels)
ax6.legend(loc='upper right')
ax6.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# --------------------------------------------------------
# PASO 5: CASOS EXTREMOS E INTERESANTES
# --------------------------------------------------------
print("\n5️⃣ CASOS EXTREMOS E INTERESANTES:")
print("-"*40)

# Errores con alta confianza
high_conf_errors = results_df[(~results_df['correct']) & (results_df['confidence'] > 0.95)]
print(f"\n⚠️ Errores con muy alta confianza (>95%): {len(high_conf_errors)} casos")
if len(high_conf_errors) > 0:
    print("\nEjemplos:")
    for idx in high_conf_errors.head(3).index:
        row = results_df.loc[idx]
        print(f"\n  • Texto: '{row['text'][:70]}...'")
        print(f"    Real: {row['true_sentiment']}, Predicho: {row['predicted_sentiment']}")
        print(f"    Confianza: {row['confidence']:.3f}")

# Aciertos con baja confianza
low_conf_correct = results_df[(results_df['correct']) & (results_df['confidence'] < 0.6)]
print(f"\n🤔 Aciertos con baja confianza (<60%): {len(low_conf_correct)} casos")
if len(low_conf_correct) > 0:
    print("\nEjemplos:")
    for idx in low_conf_correct.head(3).index:
        row = results_df.loc[idx]
        print(f"\n  • Texto: '{row['text'][:70]}...'")
        print(f"    Sentimiento: {row['true_sentiment']}")
        print(f"    Confianza: {row['confidence']:.3f}")

print("\n" + "="*60)
print("✅ Análisis de distribución de confianza completado")

In [ ]:
# ============================================================
# Celda 14 - SELECCIÓN DE CASOS PARA SHAP Y LIME
# ============================================================
# Esta celda identifica y selecciona casos estratégicos
# para análisis detallado con herramientas de explicabilidad
# ============================================================

import pandas as pd
import numpy as np

print("🎯 SELECCIÓN ESTRATÉGICA DE CASOS PARA ANÁLISIS DE EXPLICABILIDAD")
print("="*60)

# --------------------------------------------------------
# PASO 1: DEFINIR CATEGORÍAS DE CASOS INTERESANTES
# --------------------------------------------------------
print("\n1️⃣ IDENTIFICACIÓN DE CATEGORÍAS DE CASOS:")
print("-"*40)

# Diccionario para almacenar casos seleccionados
selected_cases = {
    'high_confidence_correct': [],
    'high_confidence_errors': [],
    'low_confidence_correct': [],
    'low_confidence_errors': [],
    'borderline_cases': [],
    'false_positives': [],
    'false_negatives': [],
    'representative_positive': [],
    'representative_negative': []
}

# --------------------------------------------------------
# CATEGORÍA 1: Casos de alta confianza correctos (baseline)
# --------------------------------------------------------
high_conf_correct = results_df[
    (results_df['correct'] == True) & 
    (results_df['confidence'] > 0.95)
].sort_values('confidence', ascending=False)

selected_cases['high_confidence_correct'] = high_conf_correct.head(5).index.tolist()

print("\n✅ ALTA CONFIANZA CORRECTOS (Casos baseline):")
print(f"   • Total disponibles: {len(high_conf_correct)}")
print(f"   • Seleccionados: {len(selected_cases['high_confidence_correct'])}")

# --------------------------------------------------------
# CATEGORÍA 2: Errores con alta confianza (más preocupantes)
# --------------------------------------------------------
high_conf_errors = results_df[
    (results_df['correct'] == False) & 
    (results_df['confidence'] > 0.90)
].sort_values('confidence', ascending=False)

selected_cases['high_confidence_errors'] = high_conf_errors.head(5).index.tolist()

print("\n❌ ALTA CONFIANZA INCORRECTOS (Casos críticos):")
print(f"   • Total disponibles: {len(high_conf_errors)}")
print(f"   • Seleccionados: {len(selected_cases['high_confidence_errors'])}")

# --------------------------------------------------------
# CATEGORÍA 3: Aciertos con baja confianza
# --------------------------------------------------------
low_conf_correct = results_df[
    (results_df['correct'] == True) & 
    (results_df['confidence'] < 0.65)
].sort_values('confidence', ascending=True)

selected_cases['low_confidence_correct'] = low_conf_correct.head(5).index.tolist()

print("\n🤔 BAJA CONFIANZA CORRECTOS (Aciertos por suerte?):")
print(f"   • Total disponibles: {len(low_conf_correct)}")
print(f"   • Seleccionados: {len(selected_cases['low_confidence_correct'])}")

# --------------------------------------------------------
# CATEGORÍA 4: Errores con baja confianza
# --------------------------------------------------------
low_conf_errors = results_df[
    (results_df['correct'] == False) & 
    (results_df['confidence'] < 0.65)
].sort_values('confidence', ascending=True)

selected_cases['low_confidence_errors'] = low_conf_errors.head(5).index.tolist()

print("\n⚠️ BAJA CONFIANZA INCORRECTOS (Casos difíciles):")
print(f"   • Total disponibles: {len(low_conf_errors)}")
print(f"   • Seleccionados: {len(selected_cases['low_confidence_errors'])}")

# --------------------------------------------------------
# CATEGORÍA 5: Casos fronterizos (cerca del 50%)
# --------------------------------------------------------
borderline = results_df[
    (results_df['confidence'] >= 0.48) & 
    (results_df['confidence'] <= 0.52)
].sort_values('confidence')

selected_cases['borderline_cases'] = borderline.head(5).index.tolist()

print("\n🎯 CASOS FRONTERIZOS (Confianza ~50%):")
print(f"   • Total disponibles: {len(borderline)}")
print(f"   • Seleccionados: {len(selected_cases['borderline_cases'])}")

# --------------------------------------------------------
# CATEGORÍA 6 y 7: Falsos Positivos y Negativos
# --------------------------------------------------------
false_positives = results_df[
    (results_df['true_label'] == 0) & 
    (results_df['predicted_label'] == 1)
].sort_values('confidence', ascending=False)

false_negatives = results_df[
    (results_df['true_label'] == 1) & 
    (results_df['predicted_label'] == 0)
].sort_values('confidence', ascending=False)

selected_cases['false_positives'] = false_positives.head(5).index.tolist()
selected_cases['false_negatives'] = false_negatives.head(5).index.tolist()

print("\n📊 ANÁLISIS DE ERRORES POR TIPO:")
print(f"   • Falsos Positivos: {len(false_positives)} total, {len(selected_cases['false_positives'])} seleccionados")
print(f"   • Falsos Negativos: {len(false_negatives)} total, {len(selected_cases['false_negatives'])} seleccionados")

# --------------------------------------------------------
# CATEGORÍA 8 y 9: Casos representativos por clase
# --------------------------------------------------------
# Positivos correctamente clasificados con confianza media-alta
representative_pos_mask = (
    (results_df['true_label'] == 1) & 
    (results_df['predicted_label'] == 1) &
    (results_df['confidence'] >= 0.75) &
    (results_df['confidence'] <= 0.85)
)
representative_pos_df = results_df[representative_pos_mask]

# Solo hacer sample si hay suficientes datos
if len(representative_pos_df) > 0:
    n_samples_pos = min(5, len(representative_pos_df))
    representative_pos = representative_pos_df.sample(n_samples_pos, random_state=42)
    selected_cases['representative_positive'] = representative_pos.index.tolist()
else:
    # Si no hay casos en ese rango, tomar los más cercanos
    representative_pos_alt = results_df[
        (results_df['true_label'] == 1) & 
        (results_df['predicted_label'] == 1) &
        (results_df['confidence'] >= 0.70)
    ].head(5)
    selected_cases['representative_positive'] = representative_pos_alt.index.tolist()

# Negativos correctamente clasificados con confianza media-alta
representative_neg_mask = (
    (results_df['true_label'] == 0) & 
    (results_df['predicted_label'] == 0) &
    (results_df['confidence'] >= 0.75) &
    (results_df['confidence'] <= 0.85)
)
representative_neg_df = results_df[representative_neg_mask]

# Solo hacer sample si hay suficientes datos
if len(representative_neg_df) > 0:
    n_samples_neg = min(5, len(representative_neg_df))
    representative_neg = representative_neg_df.sample(n_samples_neg, random_state=42)
    selected_cases['representative_negative'] = representative_neg.index.tolist()
else:
    # Si no hay casos en ese rango, tomar los más cercanos
    representative_neg_alt = results_df[
        (results_df['true_label'] == 0) & 
        (results_df['predicted_label'] == 0) &
        (results_df['confidence'] >= 0.70)
    ].head(5)
    selected_cases['representative_negative'] = representative_neg_alt.index.tolist()

print("\n📌 CASOS REPRESENTATIVOS (Confianza 75-85%):")
print(f"   • Positivos representativos: {len(selected_cases['representative_positive'])} seleccionados")
print(f"   • Negativos representativos: {len(selected_cases['representative_negative'])} seleccionados")

# --------------------------------------------------------
# PASO 2: CREAR DATAFRAME CON CASOS SELECCIONADOS
# --------------------------------------------------------
print("\n2️⃣ CONSOLIDACIÓN DE CASOS SELECCIONADOS:")
print("-"*40)

# Crear lista de todos los índices únicos seleccionados
all_selected_indices = []
case_categories = []

for category, indices in selected_cases.items():
    for idx in indices:
        if idx not in all_selected_indices:
            all_selected_indices.append(idx)
            case_categories.append(category)

# Crear DataFrame con los casos seleccionados
selected_df = results_df.loc[all_selected_indices].copy()
selected_df['case_category'] = case_categories

print(f"\n📊 Resumen de selección:")
print(f"   • Total de casos únicos seleccionados: {len(selected_df)}")
print(f"   • Categorías cubiertas: {len(set(case_categories))}")

# Mostrar distribución por categoría
print("\n📋 Distribución por categoría:")
for category in selected_cases.keys():
    count = sum(1 for c in case_categories if c == category)
    if count > 0:
        print(f"   • {category}: {count} casos")

# --------------------------------------------------------
# PASO 3: PREPARAR CASOS PARA SHAP/LIME
# --------------------------------------------------------
print("\n3️⃣ PREPARACIÓN DE DATOS PARA EXPLICABILIDAD:")
print("-"*40)

# Crear estructura optimizada para análisis
explainability_cases = []

for idx in selected_df.index:
    row = selected_df.loc[idx]
    case = {
        'index': idx,
        'text': row['text'],
        'true_label': row['true_label'],
        'predicted_label': row['predicted_label'],
        'confidence': row['confidence'],
        'prob_negative': row['prob_negative'],
        'prob_positive': row['prob_positive'],
        'correct': row['correct'],
        'category': row['case_category'],
        'text_length': len(row['text']),
        'word_count': len(row['text'].split())
    }
    explainability_cases.append(case)

# Convertir a DataFrame para fácil manipulación
explainability_df = pd.DataFrame(explainability_cases)

print(f"✅ Datos preparados para análisis de explicabilidad")
print(f"   • Estructura creada con {len(explainability_df)} casos")
print(f"   • Campos incluidos: {list(explainability_df.columns)}")

# --------------------------------------------------------
# PASO 4: MOSTRAR EJEMPLOS DE CADA CATEGORÍA
# --------------------------------------------------------
print("\n4️⃣ EJEMPLOS DE CASOS SELECCIONADOS POR CATEGORÍA:")
print("-"*40)

# Mostrar 1-2 ejemplos de cada categoría principal
categories_to_show = [
    'high_confidence_errors',
    'borderline_cases',
    'false_positives',
    'false_negatives'
]

for category in categories_to_show:
    cases_in_category = explainability_df[explainability_df['category'] == category]
    if len(cases_in_category) > 0:
        print(f"\n📌 {category.upper().replace('_', ' ')}:")
        for i, (idx, case) in enumerate(cases_in_category.head(2).iterrows()):
            print(f"\n  Caso {i+1}:")
            print(f"    • Texto: '{case['text'][:80]}...'")
            print(f"    • Real: {case['true_label']}, Predicho: {case['predicted_label']}")
            print(f"    • Confianza: {case['confidence']:.3f}")
            print(f"    • Prob [Neg/Pos]: [{case['prob_negative']:.3f}/{case['prob_positive']:.3f}]")

# --------------------------------------------------------
# PASO 5: GUARDAR SELECCIÓN PARA USO POSTERIOR
# --------------------------------------------------------
print("\n5️⃣ GUARDADO DE SELECCIÓN:")
print("-"*40)

# Crear diccionario con los textos para fácil acceso
selected_texts = {
    category: [results_df.loc[idx]['text'] for idx in indices if idx in results_df.index]
    for category, indices in selected_cases.items()
}

# Función helper para obtener casos específicos
def get_cases_for_explanation(category=None, n=3):
    """
    Obtiene casos específicos para análisis con SHAP/LIME
    
    Args:
        category: Categoría específica o None para obtener variados
        n: Número de casos a retornar
    
    Returns:
        Lista de tuplas (texto, índice, información)
    """
    if category and category in selected_cases:
        indices = selected_cases[category][:n]
    else:
        # Selección variada
        indices = []
        for cat, idx_list in selected_cases.items():
            if idx_list:
                indices.append(idx_list[0])
            if len(indices) >= n:
                break
    
    cases = []
    for idx in indices:
        if idx in results_df.index:
            row = results_df.loc[idx]
            cases.append({
                'text': row['text'],
                'index': idx,
                'true_label': row['true_label'],
                'predicted_label': row['predicted_label'],
                'confidence': row['confidence'],
                'category': next((cat for cat, idx_list in selected_cases.items() if idx in idx_list), 'unknown')
            })
    
    return cases

# Ejemplo de uso
example_cases = get_cases_for_explanation(category='high_confidence_errors', n=2)
print(f"\n✅ Función helper creada: get_cases_for_explanation()")
print(f"   • Ejemplo: {len(example_cases)} casos obtenidos de 'high_confidence_errors'")

print("\n" + "="*60)
print("📦 VARIABLES DISPONIBLES PARA SHAP/LIME:")
print("  • selected_df: DataFrame con todos los casos seleccionados")
print("  • explainability_df: DataFrame optimizado para análisis")
print("  • selected_cases: Diccionario con índices por categoría")
print("  • selected_texts: Diccionario con textos por categoría")
print("  • get_cases_for_explanation(): Función para obtener casos específicos")
print("\n✅ Casos listos para análisis de explicabilidad con SHAP y LIME")

In [ ]:
# ============================================================
# Celda 15 - GUARDAR SELECCIÓN Y PREPARAR PARA LIME/SHAP
# ============================================================
# Esta celda guarda los casos seleccionados y prepara todo
# para el análisis de explicabilidad
# ============================================================

import pickle
import json
import os
from datetime import datetime

print("💾 GUARDADO Y PREPARACIÓN PARA ANÁLISIS DE EXPLICABILIDAD")
print("="*60)

# --------------------------------------------------------
# PASO 1: CREAR DIRECTORIO PARA GUARDAR RESULTADOS
# --------------------------------------------------------
print("\n1️⃣ PREPARANDO ESTRUCTURA DE ARCHIVOS:")
print("-"*40)

# Crear directorio si no existe
output_dir = "explainability_analysis"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"✅ Directorio creado: {output_dir}/")
else:
    print(f"📁 Usando directorio existente: {output_dir}/")

# Timestamp para versionado
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# --------------------------------------------------------
# PASO 2: GUARDAR CASOS SELECCIONADOS
# --------------------------------------------------------
print("\n2️⃣ GUARDANDO CASOS SELECCIONADOS:")
print("-"*40)

# Guardar el DataFrame con casos seleccionados
csv_path = f"{output_dir}/selected_cases_{timestamp}.csv"
selected_df.to_csv(csv_path, index=True)
print(f"✅ DataFrame guardado: {csv_path}")
print(f"   • {len(selected_df)} casos guardados")

# Guardar diccionario de casos como JSON
json_path = f"{output_dir}/cases_by_category_{timestamp}.json"
# Convertir a formato serializable
cases_for_json = {
    category: {
        'indices': indices,
        'count': len(indices),
        'texts': [results_df.loc[idx]['text'][:200] if idx in results_df.index else None for idx in indices]
    }
    for category, indices in selected_cases.items()
}

with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(cases_for_json, f, ensure_ascii=False, indent=2)
print(f"✅ Categorías guardadas: {json_path}")

# --------------------------------------------------------
# PASO 3: CREAR ARCHIVO DE CONFIGURACIÓN PARA LIME/SHAP
# --------------------------------------------------------
print("\n3️⃣ CREANDO CONFIGURACIÓN PARA EXPLICABILIDAD:")
print("-"*40)

# Seleccionar casos prioritarios para análisis (top 3 de cada categoría importante)
priority_cases = []

priority_categories = [
    'high_confidence_errors',    # Más críticos
    'borderline_cases',          # Más inciertos
    'false_positives',           # Errores tipo I
    'false_negatives',           # Errores tipo II
    'high_confidence_correct'    # Baseline
]

for category in priority_categories:
    if category in selected_cases:
        for idx in selected_cases[category][:3]:  # Top 3 de cada categoría
            if idx in results_df.index:
                row = results_df.loc[idx]
                priority_cases.append({
                    'index': idx,
                    'category': category,
                    'text': row['text'],
                    'true_label': int(row['true_label']),
                    'predicted_label': int(row['predicted_label']),
                    'confidence': float(row['confidence']),
                    'correct': bool(row['correct'])
                })

# Guardar casos prioritarios
priority_path = f"{output_dir}/priority_cases_{timestamp}.json"
with open(priority_path, 'w', encoding='utf-8') as f:
    json.dump(priority_cases, f, ensure_ascii=False, indent=2)
print(f"✅ Casos prioritarios guardados: {priority_path}")
print(f"   • {len(priority_cases)} casos prioritarios seleccionados")

# --------------------------------------------------------
# PASO 4: CREAR FUNCIÓN DE PREDICCIÓN PARA LIME/SHAP
# --------------------------------------------------------
print("\n4️⃣ PREPARANDO FUNCIÓN DE PREDICCIÓN PARA LIME/SHAP:")
print("-"*40)

def predict_proba_for_lime(texts):
    """
    Función de predicción compatible con LIME.
    
    Args:
        texts: Lista de textos a predecir
    
    Returns:
        Array de probabilidades [prob_neg, prob_pos] para cada texto
    """
    if isinstance(texts, str):
        texts = [texts]
    
    # Hacer predicciones con el modelo
    predictions = model.predict_batch(list(texts))
    
    # Extraer probabilidades en formato LIME
    proba_array = []
    for pred in predictions:
        prob_neg = pred['probabilities']['NEGATIVE']
        prob_pos = pred['probabilities']['POSITIVE']
        proba_array.append([prob_neg, prob_pos])
    
    return np.array(proba_array)

# Verificar que funciona
test_text = "This is a test sentence."
test_proba = predict_proba_for_lime(test_text)
print(f"✅ Función de predicción creada y verificada")
print(f"   • Input test: '{test_text}'")
print(f"   • Output shape: {test_proba.shape}")
print(f"   • Probabilidades: [Neg={test_proba[0][0]:.3f}, Pos={test_proba[0][1]:.3f}]")

# --------------------------------------------------------
# PASO 5: CREAR SCRIPT DE EJEMPLO PARA LIME
# --------------------------------------------------------
print("\n5️⃣ CREANDO SCRIPT DE EJEMPLO PARA LIME:")
print("-"*40)

lime_script = f"""
# Script de ejemplo para análisis con LIME
# Generado: {timestamp}

import lime
from lime.lime_text import LimeTextExplainer
import numpy as np

# Configurar LIME
explainer = LimeTextExplainer(
    class_names=['NEGATIVE', 'POSITIVE'],
    split_expression=r'\\s+',  # Separar por espacios
    random_state=42
)

# Casos prioritarios para analizar
priority_cases = {priority_cases[:2]}  # Primeros 2 casos

# Analizar cada caso
for case in priority_cases:
    text = case['text']
    true_label = case['true_label']
    predicted_label = case['predicted_label']
    
    print(f"\\nAnalizando: '{{text[:100]}}...'")
    print(f"Real: {{true_label}}, Predicho: {{predicted_label}}")
    
    # Generar explicación
    explanation = explainer.explain_instance(
        text,
        predict_proba_for_lime,  # Tu función de predicción
        num_features=10,         # Top 10 palabras más importantes
        num_samples=500          # Número de perturbaciones
    )
    
    # Mostrar resultados
    print("\\nPalabras más importantes:")
    for word, importance in explanation.as_list():
        print(f"  • {{word}}: {{importance:.4f}}")
"""

# Guardar script
script_path = f"{output_dir}/lime_example_script.py"
with open(script_path, 'w', encoding='utf-8') as f:
    f.write(lime_script)
print(f"✅ Script de ejemplo guardado: {script_path}")

# --------------------------------------------------------
# PASO 6: RESUMEN DE ARCHIVOS CREADOS
# --------------------------------------------------------
print("\n6️⃣ RESUMEN DE ARCHIVOS CREADOS:")
print("-"*40)

print(f"\n📁 Directorio: {output_dir}/")
print(f"  • selected_cases_{timestamp}.csv - DataFrame completo")
print(f"  • cases_by_category_{timestamp}.json - Casos organizados por categoría")
print(f"  • priority_cases_{timestamp}.json - Casos prioritarios para LIME/SHAP")
print(f"  • lime_example_script.py - Script de ejemplo")

# --------------------------------------------------------
# PASO 7: VERIFICAR INSTALACIÓN DE LIME/SHAP
# --------------------------------------------------------
print("\n7️⃣ VERIFICACIÓN DE HERRAMIENTAS DE EXPLICABILIDAD:")
print("-"*40)

tools_status = {}

# Verificar LIME
try:
    import lime
    tools_status['LIME'] = f"✅ Instalado (versión {lime.__version__})"
except ImportError:
    tools_status['LIME'] = "❌ No instalado - Ejecuta: pip install lime"

# Verificar SHAP
try:
    import shap
    tools_status['SHAP'] = f"✅ Instalado (versión {shap.__version__})"
except ImportError:
    tools_status['SHAP'] = "❌ No instalado - Ejecuta: pip install shap"

for tool, status in tools_status.items():
    print(f"  • {tool}: {status}")

# --------------------------------------------------------
# RESUMEN FINAL
# --------------------------------------------------------
print("\n" + "="*60)
print("✅ PREPARACIÓN COMPLETADA")
print("\n📊 Resumen de casos guardados:")
total_cases = sum(len(indices) for indices in selected_cases.values())
print(f"  • Total de casos únicos: {len(selected_df)}")
print(f"  • Casos prioritarios: {len(priority_cases)}")
print(f"  • Categorías cubiertas: {len(selected_cases)}")

print("\n🚀 PRÓXIMOS PASOS:")
print("  1. Instalar LIME/SHAP si no están instalados")
print("  2. Usar predict_proba_for_lime() como función de predicción")
print("  3. Ejecutar análisis con los casos prioritarios")
print("  4. Visualizar explicaciones para cada categoría")

print("\n📦 VARIABLES DISPONIBLES:")
print("  • predict_proba_for_lime(): Función compatible con LIME/SHAP")
print("  • priority_cases: Lista de casos más importantes")
print("  • selected_df: DataFrame con todos los casos")
print(f"  • Archivos en: {output_dir}/")